## Import Packages and Load Data

In [2]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from pyproj import Proj, transform

In [3]:
#Load Data
# I did not include data file in Github, since it is too large
# Link: https://www150.statcan.gc.ca/n1/pub/82-402-x/2018001/hrbf-flrs-eng.htm
fp = "/Users/lengjincheng/Desktop/多大/大三study/Datafest 2020.6/Data-Fest-Covid19/postal code/HR_000a18a_e/HR_000a18a_e.shp"
data_all = gpd.read_file(fp)

In [9]:
#Load Data
data_all.head()

,HR_UID,ENGNAME,FRENAME,SHAPE_AREA,SHAPE_LEN,geometry
0,3537,City of Hamilton Health Unit,Circonscription sanitaire de la citÃ© de Hamilton,1.212619e+09,173963.691410,"POLYGON ((7192649.137 886943.691, 7192658.017 ..."
1,3538,Hastings and Prince Edward Counties Health Unit,Circonscription sanitaire des comtÃ©s de Hasti...,9.131935e+09,569876.529284,"POLYGON ((7316967.506 1154890.497, 7317200.154..."
2,3539,Huron County Health Unit,Circonscription sanitaire du comtÃ© de Huron,3.696168e+09,335661.730273,"POLYGON ((7090670.611 933316.557, 7090784.903 ..."
3,3540,Chatham-Kent Health Unit,Circonscription sanitaire de Chatham-Kent,3.078456e+09,269377.557896,"POLYGON ((7043401.120 779907.031, 7043467.977 ..."
4,3541,"Kingston, Frontenac and Lennox and Addington H...","Circonscription sanitaire de Kingston, Fronten...",8.174790e+09,460742.315962,"POLYGON ((7393218.274 1143235.086, 7393881.971..."


## Map All restaurants to a certain region

In [10]:
# File "Business ID Canada", include all data we have for restaraunts in Canada
data_id = pd.read_csv("/Users/lengjincheng/Desktop/多大/大三study/Datafest 2020.6/Data-Fest-Covid19/postal code/Business ID Canada.csv")
data_id.head()

,business_id,latitude,longitude
0,eBEfgOPG7pvFhb2wcG9I7w,44.010962,-79.448677
1,lu7vtrp_bE9PnxWfA8g4Pg,43.820492,-79.398466
2,9sRGfSVEfLhN_km60YruTA,43.947011,-79.454862
3,pcaQDBM6r0PWTXfYZK6RdA,45.425645,-73.611934
4,LoRef3ChgZKbxUio-sHgQg,45.494870,-73.651904


In [84]:
#Get health region as well as code
data_id = pd.read_csv("/Users/lengjincheng/Desktop/多大/大三study/Datafest 2020.6/Data-Fest-Covid19/postal code/Business ID Canada.csv")
health_region_id = []
health_region = []
for index, row in data_id.iterrows():
    cur_lat = row['latitude']
    cur_long = row['longitude']
    inProj = Proj(init = 'epsg:4326')
    outProj = Proj(init = 'epsg:3347')
    x,y = transform(inProj,outProj,cur_long,cur_lat) # latitude,longtitude顺序看清
    location = Point(x,y) # location
    contain = 0
    for index2, row2 in data_all.iterrows():
        if data_all.loc[index2,"geometry"].contains(location):
            health_region_id.append(data_all.loc[index2,"HR_UID"])
            health_region.append(data_all.loc[index2,"ENGNAME"])
            contain = 1
            break
    if contain == 0:
        health_region.append('None')
data_id["health_region_id"] = health_region_id
data_id["health_region_name"] = health_region

In [102]:
# import number of cases data, file name "Health_Region_Summaries Canada"
# Cases as off June 4th
data_case = pd.read_csv("/Users/lengjincheng/Desktop/多大/大三study/Datafest 2020.6/Data-Fest-Covid19/postal code/Health_Region_Summaries.csv")
data_case1 = data_case.set_index("HR_UID")
cases = []
deaths = []
for index, row in data_id.iterrows():
    region_id = int(row["health_region_id"])
    case = data_case1.loc[region_id,"CaseCount"]
    death = data_case1.loc[region_id,"Deaths"]
    cases.append(case)
    deaths.append(death)
data_id["Cases"] = cases
data_id["Deaths"] = deaths
# data_id saves all those number of cases

In [105]:
data_id.to_csv("Canada Cases and Deaths Info.csv")

## Map All regions of US

In [6]:
# American data
# I did not include data file in Github, since it is too large
# Link: https://catalog.data.gov/dataset/tiger-line-shapefile-2017-nation-u-s-current-county-and-equivalent-national-shapefile
data_all_us = gpd.read_file("/Users/lengjincheng/Desktop/多大/大三study/Datafest 2020.6/Data-Fest-Covid19/postal code/tl_2017_us_county/tl_2017_us_county.shp")

In [7]:
#clean data,get ID for each county
data_all_us["FIPS"] = data_all_us["STATEFP"]+ data_all_us["COUNTYFP"]

In [8]:
data_all_us.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,FIPS
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477641638,10701538,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",31039
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680956787,61588406,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2...",53069
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016761648,29147345,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99757, -104.56772 33.9...",35011
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169252486,22867561,+40.7835474,-096.6886584,"POLYGON ((-96.91060 40.95841, -96.91060 40.958...",31109
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645186,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089...",31129


In [ ]:
data_id_us = pd.read_csv("/Users/lengjincheng/Desktop/多大/大三study/Datafest 2020.6/Data-Fest-Covid19/postal code/tl_2017_us_county/Business_ID US.csv")
#Get health region as well as code
health_region_id_us = []
health_region_us = []
count = 0
for index, row in data_id_us.iterrows():
    cur_lat = row['latitude']
    cur_long = row['longitude']
    inProj = Proj(init = 'epsg:4326')
    outProj = Proj(init = 'epsg:4269') # american code
    x,y = transform(inProj,outProj,cur_long,cur_lat) # latitude,longtitude顺序看清
    location = Point(x,y) # location
    contain = 0
    for index2, row2 in data_all_us.iterrows():
        if data_all_us.loc[index2,"geometry"].contains(location):
            health_region_id_us.append(data_all_us.loc[index2,"FIPS"])
            health_region_us.append(data_all_us.loc[index2,"NAME"])
            contain = 1
            break
    count += 1
    print(count)
    if contain == 0:
        health_region_us.append('None')
        health_region_id_us.append('None')
data_id_us["health_region_id"] = health_region_id_us
data_id_us["health_region_name"] = health_region_us

In [125]:
# delete data with NONE
data_id_us[data_id_us.health_region_name == "None"]
data_id_us1 = data_id_us.drop([19794,22562],axis = 0)

In [127]:
# import number of cases data, file name "US County Cases"
# Cases as off June 4th
data_case_us = pd.read_csv("/Users/lengjincheng/Desktop/多大/大三study/Datafest 2020.6/Data-Fest-Covid19/postal code/tl_2017_us_county/us_county_case.csv")
data_case_us1 = data_case_us.set_index("fips")
cases_us = []
deaths_us = []
for index, row in data_id_us1.iterrows():
    region_id = int(row["health_region_id"])
    case = data_case_us1.loc[region_id,"case"]
    death = data_case_us1.loc[region_id,"death"]
    cases_us.append(case)
    deaths_us.append(death)
data_id_us1["Cases"] = cases_us
data_id_us1["Deaths"] = deaths_us
# data_id saves all those number of cases

In [129]:
data_id_us1.to_csv("US cases and death.csv")

## Join DataSet

In [11]:
data_current = pd.read_csv("/Users/lengjincheng/Desktop/多大/大三study/Datafest 2020.6/Data-Fest-Covid19/最终feature/total_business.csv",encoding = "ISO-8859-1")

/Users/lengjincheng/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,89,90,91,92,93,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
data_current.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,categories_Sports Bars,categories_Asian Fusion,categories_Mediterranean,total_checkIn,checkIn_2019,chain,covid_cumul_case_state,covid_cumul_death_state,dayOfStart,land_price
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,...,False,False,FALSE,3.0,0,FALSE,127335,5898,2658,196560
1,M_guz7Dj7hX0evS672wIwA,Chocolate Shoppe Ice Cream,2831 Parmenter St,Middleton,WI,53562,43.105310,-89.510142,3.5,6,...,False,False,FALSE,13.0,5,TRUE,20701,646,774,70070
2,mKTq1T_IAplDpHUcMzOXkw,Tipsy Ryde,NaN,Gastonia,NC,28054,35.252842,-81.152698,3.5,3,...,False,False,FALSE,NaN,None,FALSE,34809,1020,None,38976
3,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,...,False,False,FALSE,1.0,1,TRUE,32033,2484,522,11225
4,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,...,False,False,FALSE,8.0,0,FALSE,32033,2484,2020,11225


In [ ]:
data_totalcase = pd.concat([data_id, data_id_us1])
data_totalcase1 = data_totalcase[["business_id","Cases","Deaths"]]

In [142]:
data_totalcase1

,business_id,Cases,Deaths
0,eBEfgOPG7pvFhb2wcG9I7w,2650.0,222.0
1,lu7vtrp_bE9PnxWfA8g4Pg,2650.0,222.0
2,9sRGfSVEfLhN_km60YruTA,2650.0,222.0
3,pcaQDBM6r0PWTXfYZK6RdA,26288.0,3068.0
4,LoRef3ChgZKbxUio-sHgQg,26288.0,3068.0
...,...,...,...
34203,AlwKuzyrrxlZ04ktebwmcA,824.0,29.0
34204,cmsAFAymfVoBitSF8R6GKw,12761.0,489.0
34205,YZeUH6zYS0dq5QHLYZhUnQ,12761.0,489.0
34206,hskVqZCPqy-omm9CHi44xQ,5358.0,106.0


In [143]:
# Merge to get a more complete dataset
df_nostatus = pd.merge(left=data_totalcase1, right=data_current , left_on='business_id', right_on='business_id')

In [145]:
#Ready to Join with status
df_nostatus.to_csv("no_status, ready to join.csv",index = None)